<h1 align=center><font size = 5>Where could I open my new pizzeria?</font></h1>

## Introduction/Business Problem

<p style='text-align: justify;'>The problem that I want to tackle with my capstone project is to determine which neighborhood in a given city would be the best to open a new business. In order to reduce the scope of the problem, given  I've decided to restrict it to opening a new pizza parlor, that might even be something of interest for big food companies such as Pizza Hut and Domino's. <br>
The way that I've envisioned for such endeavor would be to cluster neighborhoods based on the number of pizzerias and look for clusters where there aren't that many pizzerias. </p>

## Data
<p style='text-align: justify;'>In order to tackle this problem that data that would be required would be a dataframe containing every single neighborhood in a given city, with their names and by using the GeoPy service it would be possible to acquire their latitude and longitude, with those all that's left is to query the foursquare API in order to </p>
 